# Introduction to STAC Part 2:

In this tutorial, we will highlight how to create a simple STAC catalog using the python library PySTAC. We will refer back to the catalog of items we obtained from our Planet order and the corresponding analytic assets (our COGs stored in our Google Cloud Storage bucket).

### Import Dependencies

In [1]:
# Import python packages
import pystac
import os
import json
import requests
import urllib.request
import rasterio
from tempfile import TemporaryDirectory
from pathlib import Path
import typing

# Define core repository directories
ROOT_DIR = Path(os.getcwd()).parents[2]
ASSETS_DIR = os.path.join(ROOT_DIR, "assets/")

### Initialize Catalog

In [7]:
catalog = pystac.Catalog(id='overstory-stac', description='Overstory STAC Catalog.')
print(list(catalog.get_all_items()))
print(list(catalog.get_children()))

[]
[]


### Read `manifest.json` Metadata

In [36]:
item_ids = [
         '20210425_100412_ssc3_u0001',
         '20210425_100412_ssc3_u0002',
]
    
# Helper function to read .json files
def load_item_metadata(file):
    with open(file, "r") as f:
        data = json.load(f)
        return data

# Loop through each item
for item_id in item_ids:
    
    manifest_file = os.path.join(ASSETS_DIR, item_id, 'files', 'SkySatCollect', item_id, f'{item_id}_metadata.json') 
    manifest_data = dict(load_item_metadata(manifest_file))
    
    print(manifest_data['id'])
    print(manifest_data['properties'])

20210425_100412_ssc3_u0001
{'acquired': '2021-04-25T10:04:12.75Z', 'clear_confidence_percent': 91, 'clear_percent': 93, 'cloud_cover': 0, 'cloud_percent': 0, 'ground_control_ratio': 1, 'gsd': 0.81, 'heavy_haze_percent': 0, 'item_type': 'SkySatCollect', 'light_haze_percent': 0, 'pixel_resolution': 0.5, 'provider': 'skysat', 'published': '2021-04-25T14:18:33Z', 'publishing_stage': 'finalized', 'quality_category': 'standard', 'satellite_azimuth': 35.7, 'satellite_id': 'SSC3', 'shadow_percent': 3, 'snow_ice_percent': 0, 'strip_id': 's103_20210425T100412Z', 'sun_azimuth': 145.6, 'sun_elevation': 51.7, 'updated': '2021-09-11T06:44:43Z', 'view_angle': 28.3, 'visible_confidence_percent': 78, 'visible_percent': 96}
20210425_100412_ssc3_u0002
{'acquired': '2021-04-25T10:04:12.75Z', 'clear_confidence_percent': 91, 'clear_percent': 93, 'cloud_cover': 0, 'cloud_percent': 0, 'ground_control_ratio': 1, 'gsd': 0.8, 'heavy_haze_percent': 0, 'item_type': 'SkySatCollect', 'light_haze_percent': 0, 'pixel_

In [20]:
ASSETS_DIR

'/home/alex/github_repos/ml-notebooks/assets/'

### Read Items

In [ ]:
def store_item_metadata(order_id, metadata_folder_name, item_type, item_ids):
    """Store items for metadata.
    
    Args:
        order_id (): example "20210425_100412_ssc3_u0001"
    
    """
   
    # Get previous Planet order results with the order id associated with the order
    current_order = planet_orders_url + '/' + order_id
    order_response = requests.get(current_order, auth=planet_auth)
    order_response.status_code
    order_results = order_response.json()['_links']['results']
 

    # List of metadata json file names from the selected items from the order
    file_names = [file_name := order_id + '/' +  item_type + '/' + item_id + '_metadata.json' for item_id in item_ids]

    
    # Create folder to store metadata if the folder does not already exist
    metadata_folder = Path.cwd() / metadata_folder_name
    try:
        metadata_folder.mkdir(parents=True, exist_ok=False)
    except FileExistsError as err:
        print("Directory already exists")
    
    file_paths = []
    for result in order_results:
        if result['name'] in file_names:
            json_file = result['location']
            metadata_path = Path.joinpath(metadata_folder).joinpath(result["name"])   
            file_paths.append(str(metadata_path))
            metadata_path.parent.mkdir(parents=True, exist_ok=True)
            item_metadata_download = requests.get(json_file, allow_redirects=True)
            open(metadata_path, 'wb').write(item_metadata_download.content)

In [13]:
import requests
from requests.auth import HTTPBasicAuth
import json

## Get Planet API key stored as an environment variable on your system
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

## HTTP Basic Authentication
planet_auth = HTTPBasicAuth(PLANET_API_KEY, '')

# set content type to json
headers = {'content-type': 'application/json'}

In [14]:
# Base Order URL
planet_orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# order id
order_id = 'cb363b75-b4e4-46f1-8d32-0ef87afb6215'

In [16]:
item_ids = [
         '20210515_145754_03_245c',
         '20210514_145807_70_2455',
]

## Microsoft Dask

In [6]:
import numpy as np
import xarray as xr

import rasterio.features
import stackstac
import pystac_client
import planetary_computer

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster

/home/alex/github_repos/ml-notebooks/.venv/lib/python3.8/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes
